# Without hamming distance 

In [15]:
import numpy as np

def cam(member_vector, search_vector):
    # Convert strings to numpy arrays
    member_vector = np.array([list(map(int, line.strip())) for line in member_vector])
    search_vector = np.array(list(map(int, search_vector.strip())))

    # Initialize CAM result
    cam_result = None
    
    # Iterate through each row of the member vector
    for idx, row in enumerate(member_vector):
        # Check if the search vector matches the row in the member vector
        if np.array_equal(row, search_vector):
            cam_result = idx  # If match found, set the CAM result to the index
            break  # Break the loop since we found a match
    
    return cam_result

In [17]:
member_vector = ["110000","000011","001100"]
# Define the search vector
search_vector = "001100"
# Perform CAM
result = cam(member_vector, search_vector)
# Print the CAM result
if result is not None:
    print("CAM Match found at index:", result)
else:
    print("No match found in the member vector.")


CAM Match found at index: 2


# Using Hamming based recalling 

In [18]:
def hamming_distance(pattern1, pattern2):
    return np.sum(pattern1 != pattern2)

def cam(member_vector, search_vector):
    # Convert strings to numpy arrays
    member_vector = np.array([list(map(int, line.strip())) for line in member_vector])
    search_vector = np.array(list(map(int, search_vector.strip())))

    # Initialize CAM result
    min_distance = float('inf')
    cam_result = None

    # Iterate through each row of the member vector
    for idx, row in enumerate(member_vector):
        # Calculate Hamming distance between search vector and current row
        distance = hamming_distance(row, search_vector)
        if distance < min_distance:
            min_distance = distance
            cam_result = idx

    return cam_result

member_vector = ["110000","000011","001100"]
# Define the search vector
search_vector = "001000"
# Perform CAM
result = cam(member_vector, search_vector)
# Print the CAM result
if result is not None:
    print("CAM Match found at index:", result)
else:
    print("No match found in the member vector.")

recall_vector=member_vector[result]
print(recall_vector)

CAM Match found at index: 2
001100


# Bidirectional Associative Memory

In [5]:
import numpy as np

n = 5  # Dimension of Fx
p = 4  # Dimension of Fy
q = 2  # Number of associations

# Specify Fx vectors
x = np.array([[0, 1, 0, 1, 0], [1, 1, 0, 0, 0]])

# Specify Fy vectors
y = np.array([[1, 0, 0, 1], [0, 1, 0, 1]])

# Convert to bipolar
mem_vectors_x = 2 * x - 1
mem_vectors_y = 2 * y - 1


In [6]:
# Initialize weight matrix
wt_matrix = np.zeros((n, p))

# Compute weight matrix
for i in range(q):
    wt_matrix += np.outer(mem_vectors_x[i], mem_vectors_y[i])

# Set up time index
k = 1

# Set up probe and signals
probe = np.array([0, 1, 0, 1, 1])
signal_x = 2 * probe - 1
signal_y = np.random.choice([-1, 1], size=p)

# Initialize pattern arrays
pattern_x = [signal_x]
pattern_y = [signal_y]


In [7]:
# Initialize flag for bidirectional equilibrium
flag = 0

while flag != 1:
    # Compute activations for Fx
    act_y = np.dot(signal_x, wt_matrix)

    # Update Fy signals based on activations
    signal_y = np.where(act_y > 0, 1, -1)

    # Compare for stability
    if k > 1:
        compare_y = np.array_equal(signal_y, pattern_y[-2])
    else:
        compare_y = False

    # Store Fy pattern
    pattern_y.append(signal_y)

    # Compute activations for Fy
    act_x = np.dot(signal_y, wt_matrix.T)

    # Update Fx signals based on activations
    signal_x = np.where(act_x > 0, 1, -1)

    # Compare for stability
    if k > 1:
        compare_x = np.array_equal(signal_x, pattern_x[-2])
    else:
        compare_x = False

    # Store Fx pattern
    pattern_x.append(signal_x)

    # Check for bidirectional equilibrium
    flag = int(compare_x and compare_y)

    # Increment time
    k += 1

# Display update traces
print("Pattern X:")
print(pattern_x)
print("Pattern Y:")
print(pattern_y)

Pattern X:
[array([-1,  1, -1,  1,  1]), array([-1,  1, -1,  1, -1]), array([-1,  1, -1,  1, -1]), array([-1,  1, -1,  1, -1])]
Pattern Y:
[array([1, 1, 1, 1]), array([ 1, -1, -1,  1]), array([ 1, -1, -1,  1]), array([ 1, -1, -1,  1])]
